# **Workshop #2**

### *EDA - `the_grammy_awards` dataset*
---

## ***Setting the project directory***
This script attempts to change the current working directory to the specified path.
If the directory change fails due to the directory not being found, it prints a message indicating that the user is already in the correct directory.

In [1]:
import os

try:
    os.chdir("../../Workshop #2")
except FileNotFoundError:
    print("You are already in the correct directory.")

## ***Importing dependencies***

**Modules:**
* **src.database.db_operations**: Custom module for database operations.

**For this environment we are using:**
* ***Pandas*** >= 2.2.2
* ***matplotlib*** >= 3.9.2
* ***seaborn*** >= 0.13.2

**From the src.db module, we are also using:**
* ***SQLAlchemy*** >= 2.0.32
    * *SQLAlchemy Utils* >= 0.41.2
* ***python-dotenv*** >= 1.0.1

In [2]:
from src.database.db_operations import creating_engine, disposing_engine

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

## ***Reading the data***

In this code block we are in charge of creating the database connection engine. The `creating_engine` function comes from the `src.database.db_operations` module.

In [3]:
engine = creating_engine()

15/09/2024 11:29:48 PM Engine created. You can now connect to the database.


To fetch the data we must read the *grammy_awards_raw* table coming from our PostgreSQL database.

In [4]:
grammys_data = pd.read_sql_table("grammy_awards_raw", engine)
grammys_data.head()

,year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
0,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi...",https://www.grammy.com/sites/com/files/styles/...,True
1,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V...",https://www.grammy.com/sites/com/files/styles/...,True
2,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ...",https://www.grammy.com/sites/com/files/styles/...,True
3,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H...",https://www.grammy.com/sites/com/files/styles/...,True
4,2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C...",https://www.grammy.com/sites/com/files/styles/...,True


## ***Data preparation***
---

### **Reviewing the dataset**

The Grammy dataset contains a total of 4.810 entries with 10 columns. The columns provide the following information:

**String columns (object type):**
- `title`: Title of the Grammy nomination.
- `published_at`: Date the nomination was published.
- `updated_at`: Date the nomination was last updated.
- `category`: The Grammy category.
- `nominee`: The person or group nominated.
- `artist`: The artist involved (if applicable).
- `workers`: People involved in the production.
- `img`: URL for an image related to the nomination.

**Numerical columns (int64 type):**
- `year`: The year of the Grammy nomination.

**Boolean column (bool type):**
- `winner`: Indicates if the nominee won the Grammy.

In [17]:
df = grammys_data.copy()

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4810 entries, 0 to 4809
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   year          4810 non-null   int64 
 1   title         4810 non-null   object
 2   published_at  4810 non-null   object
 3   updated_at    4810 non-null   object
 4   category      4810 non-null   object
 5   nominee       4804 non-null   object
 6   artist        2970 non-null   object
 7   workers       2620 non-null   object
 8   img           3443 non-null   object
 9   winner        4810 non-null   bool  
dtypes: bool(1), int64(1), object(8)
memory usage: 343.0+ KB


### **Checking the unique and null values**

Looking at the results, we noticed something strange in the number of unique values of *winner*, who has only 1 value. This finding will be analyzed in depth later.

In [19]:
df.nunique()

year              62
title             62
published_at       4
updated_at        10
category         638
nominee         4131
artist          1658
workers         2366
img             1463
winner             1
dtype: int64

It is interesting the important amount of null values for *artist* and *workers*, moreover: these columns are the most important in our dataset, because they are the ones that give the most description and value to our record. We will see later how the null values of these two columns could be handled.

In [20]:
df.isnull().sum()

year               0
title              0
published_at       0
updated_at         0
category           0
nominee            6
artist          1840
workers         2190
img             1367
winner             0
dtype: int64

### **Are these columns useful?**

#### **Published At and Updated At column**

In the analysis of the dataset I could not apply a clear use of the values of these two columns. The dates have no relation with the date of the event, or with any ephemeris related to this event. Therefore, I am going to delete them from the dataframe.

In [21]:
df["published_at"].value_counts()

published_at
2017-11-28T00:03:45-08:00    4205
2020-05-19T05:10:28-07:00     433
2018-12-06T23:48:49-08:00      86
2018-05-22T03:08:24-07:00      86
Name: count, dtype: int64

In [22]:
df["updated_at"].value_counts()

updated_at
2019-09-10T01:08:19-07:00    778
2019-09-10T01:06:11-07:00    754
2019-09-10T01:07:37-07:00    713
2019-09-10T01:06:59-07:00    681
2019-09-10T01:11:09-07:00    658
2019-09-10T01:09:02-07:00    554
2020-05-19T05:10:28-07:00    433
2017-11-28T00:03:45-08:00    108
2020-09-01T12:16:40-07:00     83
2019-09-10T01:11:48-07:00     48
Name: count, dtype: int64

#### **Image column**
Let's check if any of these images can be used or are available.

In [23]:
df.loc[0, "img"]

'https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh'

As we see in the output below, many of these images are **no longer available** to the public. This column is of no use whatsoever.

![Billie Elish]("https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh")

#### **Winner column**

We note that the only value available in this column is True, although in some editions the losers in certain categories are also included.

So little information really prevents us from differentiating the true winners from the other nominees. Therefore, this column can be dropped.

In [24]:
df["winner"].value_counts()

winner
True    4810
Name: count, dtype: int64

#### **Finally, we remove unnecessary columns**

In [25]:
df = df.drop(columns=["published_at", "updated_at", "img", "winner"])

In [26]:
df.head()

,year,title,category,nominee,artist,workers
0,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Fi..."
1,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin V..."
2,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster ..."
3,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph H..."
4,2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar C..."
